In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def sigmoid(x, mu, sigma):
    z = (x-mu) / sigma
    return 1 / (1 + np.exp(z))

In [19]:
all_scores=pd.read_csv('combined_csv.csv')
all_scores.dropna(axis=1, how='all', inplace=True)
all_scores.set_index('Unnamed: 0.1', inplace=True)
all_scores['if_vdw']=all_scores['if_X_fa_atr']+all_scores['if_X_fa_rep']
all_scores['ligand_vdw']=all_scores['ligand_fa_atr']+all_scores['ligand_fa_rep']
all_scores['carboxy_vdw']=all_scores['carboxy_fa_atr']+all_scores['carboxy_fa_rep']

In [ ]:
import re
from shutil import copy
import os

# calculate sigmoid for all_cst and computed_ddg
sig_filters=['if_vdw','total_score/len_prot','ligand_fa_sol','carboxy_vdw','totalS_rank','dihedral_constraint', 'angle_constraint', 'atom_pair_constraint']

label='final_1000'

dm=all_scores[(all_scores['NO2_dist1'] > 15)] 
print(len(dm))

temp_df=pd.DataFrame()
for i in sig_filters:
    if i!='totalS_rank':
        print(i)
        mean=dm[i].mean()
        std=dm[i].std()
        name= i + '_sig'
        temp_df[name] = dm[i].apply(lambda x: sigmoid(x, mu=mean, sigma=std))  
    else:
        print(i +'_loop2')
        name= i + '_sig'
        temp_df[name]=1 / (1 + np.exp(dm[i]))
    
        

sig_mult= temp_df['if_vdw_sig']*temp_df['total_score/len_prot_sig']* temp_df['ligand_fa_sol_sig']* temp_df['totalS_rank_sig'] * (temp_df['dihedral_constraint_sig']* temp_df['angle_constraint_sig']* temp_df['atom_pair_constraint_sig'])**(1/2)

sig_mult.plot.hist(bins=100, range=[-1, 1.5], label='sig_mult')
best_mult=sig_mult.nlargest(1000).index.tolist()


save_dir= f'saveDir/{label}'

try:
    os.mkdir(save_dir)
except:
    pass

Mlist=[]
for i in best_mult:
    bb=i.split('_')[3]
    Mlist.append('{bb}_0001/pdbs/{des}.pdb.gz'.format(bb=bb, des=i))
with open('get_top1000.txt','w') as f:
    f.write('\n'.join(Mlist))
    
for i in best_mult:
    bb=i.split('_')[3]
    cat_res=[p for p in re.findall(r'\d+',i.split('_')[2])]
    copy('<matchDesign>/pdbs/{des}.pdb.gz'.format(bb=bb, des=i), save_dir+'/{num}_{des}.pdb.gz'.format(num=best_mult.index(i),des=i))
    print('show sticks, {num}_{des} AND resi {c}+{a}'.format(num=best_mult.index(i), des=i, c=cat_res[0], a=cat_res[1]))
print('hide (hydro)')
print('show cartoon')
print('show sticks, resn UNK')